In [ ]:
import gradio as gr
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset once 
df = pd.read_csv("electric_cars.csv")
df.columns = df.columns.str.strip()
df['Model Year'] = pd.to_numeric(df['Model Year'], errors='coerce')
df['Electric Range'] = pd.to_numeric(df['Electric Range'], errors='coerce')
df['Base MSRP'] = pd.to_numeric(df['Base MSRP'], errors='coerce')
df['Electric Vehicle Type'] = df['Electric Vehicle Type'].fillna("Unknown")
df['Make'] = df['Make'].fillna("Unknown")
df['Model'] = df['Model'].fillna("Unknown")
df.dropna(subset=['Model Year'], inplace=True)

# Summary text
def get_summary_text():
    summary = {
        "🚗 Total Cars": len(df),
        "📦 Unique Models": df['Model'].nunique(),
        "🏆 Top Manufacturer": df['Make'].value_counts().idxmax(),
        "🔋 Avg Electric Range": f"{df['Electric Range'].mean():.2f} km",
        "💰 Avg MSRP": f"${df['Base MSRP'].mean():,.2f}"
    }
    return "\n".join([f"{k}: {v}" for k, v in summary.items()])

# Plot generator
def plot_eda(plot_type):
    fig, ax = plt.subplots(figsize=(10, 5))
    plt.xticks(rotation=45)

    if plot_type == "Electric Vehicle Type Count":
        sns.countplot(y='Electric Vehicle Type', data=df, order=df['Electric Vehicle Type'].value_counts().index, ax=ax)
    elif plot_type == "EVs Over Model Years":
        df['Model Year'].value_counts().sort_index().plot(kind='line', marker='o', ax=ax)
        ax.set_xlabel("Model Year"); ax.set_ylabel("Count")
    elif plot_type == "Top 10 Makes":
        top_makes = df['Make'].value_counts().head(10)
        sns.barplot(x=top_makes.values, y=top_makes.index, ax=ax)
    elif plot_type == "Top 10 Models":
        top_models = df['Model'].value_counts().head(10)
        sns.barplot(x=top_models.values, y=top_models.index, ax=ax)
    elif plot_type == "Top 10 Counties":
        top_counties = df['County'].value_counts().head(10)
        sns.barplot(x=top_counties.values, y=top_counties.index, ax=ax)
    elif plot_type == "Electric Range Distribution":
        sns.histplot(df['Electric Range'].dropna(), bins=30, kde=True, ax=ax)
    elif plot_type == "CAFV Eligibility Pie Chart":
        ax.clear()
        counts = df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].value_counts()
        ax.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=140)
        ax.axis('equal')
    elif plot_type == "Average Range by Vehicle Type":
        avg_range = df.groupby('Electric Vehicle Type')['Electric Range'].mean().sort_values()
        sns.barplot(x=avg_range.values, y=avg_range.index, ax=ax)
    elif plot_type == "Average MSRP by Vehicle Type":
        avg_msrp = df.groupby('Electric Vehicle Type')['Base MSRP'].mean().sort_values()
        sns.barplot(x=avg_msrp.values, y=avg_msrp.index, ax=ax)

    plt.tight_layout()
    return fig

# Car Finder
def car_finder(budget, min_range, min_year):
    results = df[
        (df['Base MSRP'] <= budget) &
        (df['Electric Range'] >= min_range) &
        (df['Model Year'] >= min_year)
    ][['Make', 'Model', 'Model Year', 'Electric Range', 'Base MSRP', 'Electric Vehicle Type']]
    return results.reset_index(drop=True)

# Gradio Web App
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Image(value="logo car.png", label="Electric Vehicle Dashboard Logo", show_label=True)
    
    gr.Markdown("# ⚡ Electric Vehicle Dashboard")
    gr.Markdown("Explore electric vehicle data with visual EDA tools and car finder functionality. This version uses a built-in dataset, no upload needed!")

    # Dataset Summary
    gr.Markdown("### 📊 Dataset Summary")
    gr.Textbox(value=get_summary_text(), lines=6, interactive=False)

    # EDA Section
    gr.Markdown("### 🔬 EDA Visualizations")
    plot_dropdown = gr.Dropdown(
        choices=[
            "Electric Vehicle Type Count",
            "EVs Over Model Years",
            "Top 10 Makes",
            "Top 10 Models",
            "Top 10 Counties",
            "Electric Range Distribution",
            "CAFV Eligibility Pie Chart",
            "Average Range by Vehicle Type",
            "Average MSRP by Vehicle Type"
        ],
        label="Choose Analysis Type"
    )
    eda_plot = gr.Plot()
    plot_dropdown.change(fn=plot_eda, inputs=plot_dropdown, outputs=eda_plot)

    # Car Finder Section
    gr.Markdown("### 🔍 EV Finder Tool")
    with gr.Row():
        budget = gr.Number(value=40000, label="💰 Max Budget ($)")
        min_range = gr.Number(value=200, label="🔋 Min Range (km)")
        min_year = gr.Number(value=2018, label="📅 Model Year From")
    car_output = gr.Dataframe(label="✅ Matching Cars")
    find_btn = gr.Button("Find Cars 🚀")
    find_btn.click(fn=car_finder, inputs=[budget, min_range, min_year], outputs=car_output)

# Launch with public link
demo.launch(share=True)
